In [2]:
import torch
from torch.nn import Linear, Parameter
from torch_geometric.nn import MessagePassing
from torch_geometric.utils import add_self_loops, degree

In [ ]:
class GCNConv(MessagePassing):
    def __init__(self,in_channels,out_channels):
        super().__init__(aggr='add') # 用加法做聚合方式
        self.lin=Linear(in_channels,out_channels,bias=False)
        self.bias=Parameter(torch.empty(out_channels))
        # 初始化参数
        self.reset_parameters()
    
    def reset_parameters(self):
        self.lin.reset_parameters()
        self.bias.data.zero_()

    def forward(self,x,edge_index):
        # 先加入自环
        edge_index,_=add_self_loops(edge_index,num_nodex=x.size(0))

        # 线性转化
        x=self.lin(x)

        # 正则化邻接矩阵
        row,col=edge_index
        deg=degree(col,x.size(0),dtype=x.dtype)
        deg_inv_sqrt=deg.pow(-0.5)
        deg_inv_sqrt[deg_inv_sqrt==float('inf')]=0
        # 计算边的起点和终点的总的权重
        norm=deg_inv_sqrt[row]*deg_inv_sqrt[col]

        # 进行progate
        out=self.propagate(edge_index,x=x,norm=norm)

        # 加上bias
        out+=self.bias
    
        return out
    
    def message(self,x_j,norm):
        # x_j的形状是[E,out_channels],E是所有边的数量，就是所有边的终点的的特征

        # 开始标准化D-0.5AD-0.5这个步骤，对终点做了就够了，之后会在propagate里进行edge_index和点的特征的计算
        # 转化为列向量，并使用矩阵的正常乘法
        return norm.view(-1,1) * x_j

# 我的理解
# propagate基本流程
``` python
class MessagePassing(torch.nn.Module):
    
    def propagate(self, edge_index, size=None, **kwargs):
        """
        :param edge_index: 图的边的索引，通常是一个 2 x E 的张量，其中 E 是边的数量。
        :param size: 如果图是 bipartite 的，即有两类不同的节点，这个参数用来提供每类节点的数量。
        :param kwargs: 任何其他需要传递给 `message` 和 `update` 函数的参数。
        """
        
        # 阶段 1: Message
        # 使用传递给 propagate 的参数计算消息
        msg_kwargs = self.inspector.distribute('message', kwargs)
        messages = self.message(**msg_kwargs)
        
        # 阶段 2: Aggregate
        # 使用传递给 propagate 的边的索引来聚合消息
        out = self.aggregate(messages, edge_index, size=size)
        
        # 阶段 3: Update
        # 使用聚合的消息来更新节点的特征
        update_kwargs = self.inspector.distribute('update', kwargs)
        out = self.update(out, **update_kwargs)
        
        return out

```

# aggregate简略实现
## 这里的index是edge_index[1]
```python
class MyConv(MessagePassing):
    # ...
    def aggregate(self, inputs, index, ptr=None, dim_size=None):
        # Custom aggregation logic
        pass

```
## scatter作用示例
```python
import torch
from torch_scatter import scatter

# source: 源数据，将要被聚合的数据。
source = torch.tensor([10, 20, 30, 40, 50], dtype=torch.float)

# index: 目标索引，定义了source中的数据应该如何被映射到输出中。
index = torch.tensor([0, 1, 1, 2, 2], dtype=torch.long)

# 使用scatter函数进行加和聚合。
output = scatter(source, index, reduce="sum")

print(output)

```
结果：tensor([10., 50., 90.])
解释一下这个结果：

输出的第0个元素是10，因为 source 中对应于 index 中0的元素是10。
输出的第1个元素是50，因为 source 中对应于 index 中1的元素是20和30，它们的和是50。
输出的第2个元素是90，因为 source 中对应于 index 中2的元素是40和50，它们的和是90。


# message中x_j是源节点source node，x_i是目标节点target node